#**Project Omega**

#Project's Goal
The main research goal is to set a program that will predict the gender of the author of a tweet and we could extend the model to any text.<br><br>
To do so, we will use some text analytics techniques seen in class. Those techniques are tokenization, BOW (bag of words), TF-IDF stylometrie, cosine-similarity and try to adapt some sentiment-analysis.<br><br>
In term of business, with this idea we aim to enhence marketing targeting. Third party advertisement companies can have better knowledge on Twitter’s gender distribution. Thus, they can advise companies that want to make advertisement on social network like Twitter. 

##Importation and Installation of methods
For this project, we will use different packages. The main part will be about text mining. For this, we will use packages seen in class, which are *spacy*, *nltk* and *enchant*. 

In [0]:
import nltk
!pip install spacy
!apt install -qq enchant
!pip install pyenchant
!pip install nltk
!python -m spacy download en
nltk.download('punkt')
nltk.download('wordnet')
!pip install chart_studio

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import spacy
from spacy.lang.en import English
import enchant
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
import collections
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import chart_studio.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import operator

In [0]:
#Import data
data = pd.read_csv('https://raw.githubusercontent.com/XaviJunior/omega/Data/Data/gender-classifier-DFE-791531.csv',encoding="latin-1")
textgender=data[["gender",'text',"description"]]
textgender=textgender.dropna()
X=textgender[['text','description']]
y=textgender["gender"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=14)
train= pd.concat([X_train.reset_index(drop='Tru‌​e'),y_train.reset_index(drop='Tru‌​e')],axis=1)
test=pd.concat([X_test.reset_index(drop='Tru‌​e'),y_test.reset_index(drop='Tru‌​e')],axis=1)

#Cosine Similarity
The first method we will apply, is the cosine-similarity. As we seen in class, this method is often use to do text-similarity.<br>
Our first idea, was to do the cosine similarity between all train tweets and test tweets and then classify the tweet according the most similar one. But we realised that the data set was to big and it would take too much time to go through all the test tweets. (Our code runs an entire night and was not finished) <br>
So our second idea was to merge all train tweet and the description of the account (to gain text) as two documents, one with text written by male and another by female. With that, we will be able to gain information such as most common words used by each gender.

##Data Cleaning <br>
We first began by separate tweet by gender. We made new tables, one for male and one for female. <br>
We are not interested by all the feautres (columns), we kept the "gender", the "text" and the "name" for the brand. <br>
We chose to split the data by gender to be able to construct general BOW for each gender. As a tweet is necessarily written by a male or a female (only take into account biological gender), for our train and test set, we decide to ingonre tweet written by *brand* or tweet labeled as *unknown*.


In [0]:
#New Table with Male Only and Split in Test/Train Data
Male = train[train['gender'] == 'male']
Maletext=Male['text']
MaleDes=Male['description']
ym=Male["gender"]
ListMale=Maletext.values.tolist()+MaleDes.values.tolist()

#New Table with Female Only and split in Test/Train Data
Female = train[train['gender'] == 'female']
Femaletext=Female['text']
FemaleDes=Female['description']
yf=Female["gender"]
ListFemale=Femaletext.values.tolist()+FemaleDes.values.tolist()

###Merging
Now that we create different table for both gender. We will merge all the tweets in two documents, for one male and one for female. We do that because we are interested to know which words are the most used by male and female.

In [0]:
#Merge all Male Text in a single string
TextMale=" "
for i in range(0,len(ListMale)):
  TextMale=TextMale+ListMale[i]
textmale=TextMale.lower()

#Merge all Female Text in a single string
TextFemale=" "
for i in range(0,len(ListFemale)):
  TextFemale=TextFemale+ListFemale[i]
textfemale=TextFemale.lower()

### Tokenization, Stop Words, Lemmatization
Here, we created the function that will do the text cleaning.<br>
First, the tokenization will allow us to separate each word and to be able to calulate the frequencie of each single word. The tokenization's technique chosen is the white-space one.<br>
Now that we have tokens, we want to clean the text by removing stop words and other tokens such as "\x89\x9d_\x95ü\x8f\x89\x9d_\x95ü\x8f\x89\x9d_\x95ü\x8f". To do so, we will use a stop words list available in *spacy* and filter words that are not in the english dictionnary available on *pyenchant*. We know that by using an ensglish dictionnary we may lose some expression that are used on twitter. But for us, it was the only sustainable solution to filter non-sense tokens. We will also test this function without the dictionnary to see which one gives use the best accuracy. <br>
We finish the function by doing some lemmatization.

In [0]:
#Cleaning function
def cleaning(input):
  nlp = English()
  Doc=nlp(input)
  Token = []
#Check if words in english dictionnary
  for token in Doc:
    Token.append(token.text)
  Words=[]
  d = enchant.Dict("en_US")
  Text=''
  for i in Token:
    if d.check(i)==True and i != ' ': #comment this line to try without dictionnary
      Text+=' '+i         #remove an indenatation to try without dictionnary
  doc=nlp(Text)
#Remove stop words and punctuation
  for word in doc:
    if word.is_stop==False and word.is_punct==False:
      Words.append(word)
  words=''
#Lemmatization
  for j in Words:
    lemm=lemmatizer.lemmatize(str(j))
    words+=lemm+' '
  words=words.lower()
  return(words)

In [0]:
#Tokenizer found in the DMML class lab 6.1
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

Now, we will apply this function to our two documents.

In [0]:
Fem=cleaning(textfemale)
Mal=cleaning(textmale)

##Bag Of Words (BOW)
Here we are going to create a table with a line per gender. The columns are all the words used on the train tweet of our dataset. As we merge all the tweet, it would have made no sense to set bag of words of n-gramns other than one. Because at some point some BOW would have been wirtten by two different people. We create two functions, the first is to plot the BOW for our *female* and *male* documents, the second one will plot the BOW with these documents and the tweet for which we want to predict the gender.<br>
As we only have 2 documents, doing some TF-IDF would not make a lot of sence.

In [0]:
#Creation and ploting of bag of words for two documents
def bagofwords(Fem,Mal):
  text=[Fem,Mal]
  count = CountVectorizer()
  bow = count.fit_transform(text)
  feature_names = count.get_feature_names()
  BagOfWords=pd.DataFrame(bow.todense(),columns=feature_names, index=['Female','Male'])
  return BagOfWords.head()

In [0]:
#Creation and ploting of bag of words for three documents
def BOW(clean):
  clean=cleaning(clean)
  text=[Fem,Mal,clean]
  count = CountVectorizer()
  bow = count.fit_transform(text)
  feature_names = count.get_feature_names()
  BagOfWords=pd.DataFrame(bow.todense(),columns=feature_names, index=['Female','Male','Test'])
  return BagOfWords.head()

In [0]:
bagofwords(Fem,Mal)

##Cleaning of the Test Data
This part is a little bit more technical as the first cleaning. In the first case, we merged all tweet so we had only two documents. <br>
Now, we want to clean tweet by tweet and have several clean documents. As we cleaned the *train* set, we need to apply the same method to the *test* set. The goal of this section is to build a list of *cleaned tweets* by gender. <br>
In this part, as our goal is to predict gender by tweet only, we won't take into account the account's description.

In [0]:
#Here, we will set the test set as random mix of tweet.
TestSetM = test[test['gender'] == 'male']
TestSetF = test[test['gender'] == 'female']
TestSet=TestSetM.append(TestSetF)
TestSet1=TestSet.sample(frac=1)
X_test=TestSet1["text"]
y_test=TestSet1["gender"]
Test=X_test.values.tolist()

In [0]:
Clean_Test=[]
for i in Test:
  Clean_Test.append(spacy_tokenizer(i))
Test_Clean=[]
for i in range(0, len(Clean_Test)):
  Test_Clean.append(' '.join(Clean_Test[i]))

##Classifying
Now, the funny part can begin. As explain before, the first method chosen is the cosine-similarity. To do so, we created a function that will compute the cosine-similarity of each test tweet with our two documents (female's one and male's one). The function will classify the tweet according the highest cosine-similarity. In our case, as the base rate is *female*, in case of equlality between the cosine-similarities, we classify the tweet as a *female*. The model might be changed for the final version, for that, we will check the base rate of all tweeter's users.

In [0]:
#Classification with Cosine Similarity
def classifier(clean):
  clean=cleaning(clean)
  y_pred=[]
  text=[Fem,Mal,clean]
  count = CountVectorizer()
  bow = count.fit_transform(text)
  cosine =cosine_similarity(bow[2],bow)
  if cosine[0][0]>=cosine[0][1]:
    y_pred.append("female")
  else:
    y_pred.append("male")
  return y_pred

In [0]:
#might take some time as we have a lot of data
y_pred=[]
for i in Test_Clean:
  y_pred.append(classifier(i))

###Accuracy

In [0]:
accuracy_score(y_pred,y_test)

We see that our accuracy is higher than the BaseRate of the model. It means that the accuracy will be higher by using our model than classify every tweet as written by female. But the difference is not huge. We will try another model.

###Predict Tweet's gender
Here, you can input a tweet of your choice and our model will try to predict the author's gender.


In [0]:
#I won't buy you beer, but I will bring you pleasure, here is my paypal. Don't hesitate to give me money. Do you want nudes ? I am full shaved. I will cook for you and more if you are down for it ;) As my uncle is dead, I have no cash left, help me and I will cum for you ! Oops come ;).
Tweet=input("Enter a Tweet: ")
print('This tweet was probably writtent by a:',classifier(Tweet))
BOW(Tweet)

#Logistic Regression
In this part, we will predict the gender according a logistic Regression. To do so, we will try to adapt a *Sentiment Analysis* model. The equivalent of the sentiment will be the gender. To do that, we won't use the merge documents for the training part. We will keep all tweets as an input.

##Data Cleaning
For this model, we need to clean all our tweets one by one and we won't use the description.

In [0]:
TrainSetM = train[train['gender'] == 'male']
TrainSetF = train[train['gender'] == 'female']
TrainSet=TrainSetM.append(TrainSetF)
TrainSet1=TrainSet.sample(frac=1)
X_train=TrainSet1["text"]
y_train=TrainSet1["gender"]
Train=X_train.values.tolist()

In [0]:
Clean_Train=[]
for i in Train:
  Clean_Train.append(spacy_tokenizer(i))
Train_Clean=[]
for i in range(0, len(Clean_Train)):
  Train_Clean.append(' '.join(Clean_Train[i]))

##Classifying
For this model, we will use a cleaning function that was given during our DMML class (see lab 6.1). The cleaning methods works better with this model than the functio we wrote ourself. But our function is better with the cosine-similarity than that cleaning's function.

In [0]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)
from sklearn.linear_model import LogisticRegression
classifier2 = LogisticRegression(solver="lbfgs")

# Create pipeline using Bag of Words
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier2', classifier2)])

# model generation
y_train=TrainSet1["gender"]
pipe.fit(Train,y_train)

###Accuracy

In [0]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(Test)

# Model Accuracy
print(" test Accuracy:",metrics.accuracy_score(y_test, predicted))
print(" Precision:",metrics.precision_score(y_test, predicted, average=None))
print(" Recall:",metrics.recall_score(y_test, predicted, average=None))

We see that the accuracy in this case is quite equal to the accuracy obtained with the cosine similarity.

###Predict Tweet's gender


In [0]:
Tweet=input("Enter a Tweet: ")
Tweet=[Tweet]
print('This tweet was probably written by a:',pipe.predict(Tweet))

##Decision Tree

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
texts = Train_Clean
# using default tokenizer 
count = CountVectorizer()
bow_train = count.fit_transform(texts)

In [0]:
bow_train.toarray()

feature_names = count.get_feature_names()
BoW=pd.DataFrame(
    bow_train.todense(), 
    columns=feature_names
    )

In [0]:
y_train=TrainSet1["gender"]
y_train= y_train.replace({'male':0})
y_train = y_train.replace({'female':1})
y_train=y_train.values.tolist()

In [0]:
#with n-gram 1 and 2, it might crash
BoW['Gender']=y_train
cols = list(BoW.columns)
cols = [cols[-1]] + cols[:-1]
BoW = BoW[cols]

In [0]:
BoW.head()

In [0]:
clf = DecisionTreeClassifier(criterion='entropy')

In [0]:
y_train=TrainSet1['gender']
clf.fit(bow_train, y_train)

In [0]:
texts_test = Test
# using default tokenizer 
bow_test = count.transform(texts_test)

# Show feature matrix
bow_test=bow_test.toarray()

In [0]:
clf.score(bow_test,y_test)

In [0]:
from sklearn.decomposition import TruncatedSVD
from scipy import sparse as sp
clf = TruncatedSVD(1000)
Xpca_Test = clf.fit_transform(bow_test)
Xpca_Train = clf.fit_transform(bow_train)

In [0]:
from sklearn.linear_model import LogisticRegressionCV
LR = LogisticRegressionCV(solver='lbfgs', cv=5,max_iter=1000)

In [0]:
LR.fit(Xpca_Train, y_train)

In [0]:
LR.score(Xpca_Test,y_test)

In [0]:
LR.fit(bow_train,y_train)

In [0]:
LR.score(bow_test,y_test)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

In [0]:
knn.fit(Xpca_Train,y_train)

In [0]:
knn.score(Xpca_Test,y_test)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer(ngram_range=(1, 1))
features = tfidf.fit_transform(texts)
TFIDF=pd.DataFrame(features.todense(),columns=tfidf.get_feature_names())
y_train=TrainSet1["gender"]
y_train=y_train.values.tolist()
TFIDF['Gender']=y_train
cols = list(TFIDF.columns)
cols = [cols[-1]] + cols[:-1]
TFIDF = TFIDF[cols]
TFIDF= TFIDF.replace({'male':0})
TFIDF = TFIDF.replace({'female':1})
TFIDF.head()